<a href="https://colab.research.google.com/github/rushiai/Advance-House-Price/blob/main/24_5_YT_all_text_%2B_summary_%2B_audio_%2B_images_only_video_remaining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
!git clone https://github.com/TensorSpeech/TensorFlowTTS
os.chdir("TensorFlowTTS")
!pip install  .
os.chdir("..")
import sys
sys.path.append("TensorFlowTTS/")
print("Downloading Tacotron2 model...")
!gdown --id {"12jvEO1VqFo1ocrgY9GUHF_kVcLn3QaGW"} -O tacotron2-120k.h5
!gdown --id {"1OI86hkN1YCpHBsIKnkELNbSho5Pj-pPY"} -O tacotron2_config.yml
print("Downloading MelGAN-STFT model...")
!gdown --id {"1WB5iQbk9qB-Y-wO8BU6S2TnRiu4VU5ys"} -O melgan.stft-2M.h5
!gdown --id {"1OqdrcHJvtXwNasEZP7KXZwtGUDXMKNkg"} -O melgan.stft_config.yml
print("Downloading MelGAN model...")
!gdown --id {"1A3zJwzlXEpu_jHeatlMdyPGjn1V7-9iG"} -O melgan-1M6.h5
!gdown --id {"1Ys-twSd3m2uqhJOEiobNox6RNQf4txZs"} -O melgan_config.yml
import tensorflow as tf

import yaml
import numpy as np
import matplotlib.pyplot as plt

import IPython.display as ipd

from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoConfig
from tensorflow_tts.inference import AutoProcessor

tacotron2_config = AutoConfig.from_pretrained('TensorFlowTTS/examples/tacotron2/conf/tacotron2.v1.yaml')
tacotron2 = TFAutoModel.from_pretrained(
    config=tacotron2_config,
    pretrained_path="tacotron2-120k.h5",
    name="tacotron2"
)

melgan_config = AutoConfig.from_pretrained('TensorFlowTTS/examples/melgan/conf/melgan.v1.yaml')
melgan = TFAutoModel.from_pretrained(
    config=melgan_config,
    pretrained_path="melgan-1M6.h5",
    name="melgan"
)

melgan_stft_config = AutoConfig.from_pretrained('TensorFlowTTS/examples/melgan_stft/conf/melgan_stft.v1.yaml')
melgan_stft = TFAutoModel.from_pretrained(
    config=melgan_stft_config,
    pretrained_path="melgan.stft-2M.h5",
    name="melgan_stft"
)

print("Downloading ljspeech_mapper.json ...")
!gdown --id {"1YBaDdMlhTXxsKrH7mZwDu-2aODq5fr5e"} -O ljspeech_mapper.json

processor = AutoProcessor.from_pretrained(pretrained_path="./ljspeech_mapper.json")

def do_synthesis(input_text, text2mel_model, vocoder_model, text2mel_name, vocoder_name):
  input_ids = processor.text_to_sequence(input_text)

  # text2mel part
  if text2mel_name == "TACOTRON":
    _, mel_outputs, stop_token_prediction, alignment_history = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        tf.convert_to_tensor([len(input_ids)], tf.int32),
        tf.convert_to_tensor([0], dtype=tf.int32)
    )
  elif text2mel_name == "FASTSPEECH":
    mel_before, mel_outputs, duration_outputs = text2mel_model.inference(
        input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
        speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    )
  elif text2mel_name == "FASTSPEECH2":
    mel_before, mel_outputs, duration_outputs, _, _ = text2mel_model.inference(
        tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
        speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        f0_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        energy_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
    )
  else:
    raise ValueError("Only TACOTRON, FASTSPEECH, FASTSPEECH2 are supported on text2mel_name")

  # vocoder part
  if vocoder_name == "MELGAN" or vocoder_name == "MELGAN-STFT":
    audio = vocoder_model(mel_outputs)[0, :, 0]
  elif vocoder_name == "MB-MELGAN":
    audio = vocoder_model(mel_outputs)[0, :, 0]
  else:
    raise ValueError("Only MELGAN, MELGAN-STFT and MB_MELGAN are supported on vocoder_name")

  if text2mel_name == "TACOTRON":
    return mel_outputs.numpy(), alignment_history.numpy(), audio.numpy()
  else:
    return mel_outputs.numpy(), audio.numpy()

def visualize_attention(alignment_history):
  import matplotlib.pyplot as plt

  fig = plt.figure(figsize=(8, 6))
  ax = fig.add_subplot(111)
  ax.set_title(f'Alignment steps')
  im = ax.imshow(
      alignment_history,
      aspect='auto',
      origin='lower',
      interpolation='none')
  fig.colorbar(im, ax=ax)
  xlabel = 'Decoder timestep'
  plt.xlabel(xlabel)
  plt.ylabel('Encoder timestep')
  plt.tight_layout()
#   plt.show()
  plt.close()

def visualize_mel_spectrogram(mels):
  mels = tf.reshape(mels, [-1, 80]).numpy()
  fig = plt.figure(figsize=(10, 8))
  ax1 = fig.add_subplot(311)
  ax1.set_title(f'Predicted Mel-after-Spectrogram')
  im = ax1.imshow(np.rot90(mels), aspect='auto', interpolation='none')
  fig.colorbar(mappable=im, shrink=0.65, orientation='horizontal', ax=ax1)
#   plt.show()
  plt.close()

Cloning into 'TensorFlowTTS'...
remote: Enumerating objects: 10380, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 10380 (delta 43), reused 42 (delta 17), pack-reused 10290
Receiving objects: 100% (10380/10380), 131.44 MiB | 34.81 MiB/s, done.
Resolving deltas: 100% (4983/4983), done.
Processing /content/TensorFlowTTS
     |████████████████████████████████| 320.4MB 45kB/s 
     |████████████████████████████████| 686kB 43.8MB/s 
     |████████████████████████████████| 245kB 57.8MB/s 
     |████████████████████████████████| 225kB 17.5MB/s 
     |████████████████████████████████| 3.6MB 44.8MB/s 
     |████████████████████████████████| 1.3MB 45.3MB/s 
     |████████████████████████████████| 1.7MB 46.5MB/s 
     |████████████████████████████████| 3.1MB 46.0MB/s 
  Cloning https://github.com/repodiac/german_transliterate.git to /tmp/pip-install-uxx0r34_/german-transliterate
  Running command git clone -q https://github.com/r

Downloading...
From: https://drive.google.com/uc?id=12jvEO1VqFo1ocrgY9GUHF_kVcLn3QaGW
To: /content/tacotron2-120k.h5
128MB [00:01, 109MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1OI86hkN1YCpHBsIKnkELNbSho5Pj-pPY
To: /content/tacotron2_config.yml
100% 1.49k/1.49k [00:00<00:00, 2.28MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WB5iQbk9qB-Y-wO8BU6S2TnRiu4VU5ys
To: /content/melgan.stft-2M.h5
17.1MB [00:00, 105MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1OqdrcHJvtXwNasEZP7KXZwtGUDXMKNkg
To: /content/melgan.stft_config.yml
100% 1.77k/1.77k [00:00<00:00, 2.66MB/s]
Downloading...
From: https://drive.google.com/uc?id=1A3zJwzlXEpu_jHeatlMdyPGjn1V7-9iG
To: /content/melgan-1M6.h5
17.1MB [00:00, 80.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ys-twSd3m2uqhJOEiobNox6RNQf4txZs
To: /content/melgan_config.yml
100% 1.29k/1.29k [00:00<00:00, 1.92MB/s]
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...


In [94]:
!apt update &> /dev/null
!apt install imagemagick &> /dev/null
!apt install ffmpeg &> /dev/null
!pip3 install moviepy[optional] &> /dev/null
!sed -i '/<policy domain="path" rights="none" pattern="@\*"/d' /etc/ImageMagick-6/policy.xml

In [2]:
!pip install transformers
!pip install newspaper3k
!pip install kora -q
from kora.selenium import wd
import nltk
nltk.download('punkt')

     |████████████████████████████████| 2.3MB 24.8MB/s 
     |████████████████████████████████| 901kB 35.8MB/s 
     |████████████████████████████████| 3.3MB 46.2MB/s 
     |████████████████████████████████| 215kB 26.8MB/s 
     |████████████████████████████████| 7.4MB 44.5MB/s 
     |████████████████████████████████| 92kB 11.7MB/s 
     |████████████████████████████████| 81kB 10.8MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp37-none-any.whl size=13538 sha256=33f2148bef6ffde380a83307088da0d9be7c844933c0bc69cece47ef298f0c87
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp37-none-any.whl size=7398406 sha256=0e7ee08f8f987ce9b01438caf6c3e58995002aac2be4dafa125fc8d6620d04a5
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp37-none-any.whl

True

In [85]:
pip install path

In [86]:
import time
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import os
import glob
import shutil
import numpy as np
import re
from newspaper import Article

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from transformers import pipeline

In [87]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from kora.selenium import wd
import os
import scipy
from scipy.io.wavfile import write
import glob
import shutil
import textwrap
# from resizeimage import resizeimage
from moviepy.editor import * # ImageClip
from PIL import Image
import os
from path import Path
import numpy as np
import cv2

In [4]:
summarizer = pipeline("summarization", device=0)
# summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False)

In [55]:
categories = ['b','e']
countries = ['US','CA']
master_dic = {}

In [56]:
for count in countries:
    master_dic[count] = {}
    for cat in categories:
        df = pd.DataFrame()
        dfa = pd.DataFrame()
        link = 'https://trends.google.com/trends/trendingsearches/realtime?geo={0}&category={1}'.format(count,cat)
        master_dic[count][cat] = {}
        wd.get(link)
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        page = wd.page_source
        soup = BeautifulSoup(page, 'html.parser')
        result = soup.find("article")
        head_results = soup.find_all("div", class_="summary-text")
        regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        for i in head_results:
            # print(i)
            for j in i:
                # print(j)
                linkss = re.findall(regex,str(j))
                if len(linkss) != 0:
                    # print(re.findall(regex,str(linkss)))
                    z = re.findall(regex,str(linkss))
                    k = pd.Series(z)
                    df = df.append(k,ignore_index=True)

        for i in range(0,len(df)):
            a = pd.Series(df[0][i][:-2])
            dfa = dfa.append(a,ignore_index=True)
        dfa['text'] = np.nan
        for i in range(0,len(df)):
            url = dfa[0][i]
            article = Article(url)
            try:
                article.download()
                article.parse()
                article.nlp()
                art_text = article.text
                s = pd.Series(art_text)
                # dfa['text'][i] = dfa.append(s,ignore_index=True)
                dfa['text'][i] = art_text
            except:
                print('error ',i)
        dfa = dfa.dropna()
        master_dic[count][cat] = dfa

error  0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


error  8
error  10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:6

error  6


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [57]:
master_dic

{'CA': {'b':                                                    0                                               text
  0  https://www.waterloochronicle.ca/news-story/10...                                                   
  1  https://www.foxbusiness.com/business-leaders/e...  Jurors will be able to hear limited evidence o...
  2  https://ksusentinel.com/2021/05/23/home-elevat...  Latest research on Global Home Elevator Market...
  3  http://www.stocksequity.com/active-stocks/blus...  Shares of Vaxart, Inc. (NASDAQ:VXRT) reached a...
  4  https://www.francetvinfo.fr/sciences/espace/es...  Une nouvelle étape est franchie vers le touris...
  5  https://www.baytoday.ca/local-news/need-beer-o...  The Beer Store says 75 select locations across...
  7  https://www.loshijosdelamalinche.com/trending-...  “\n\nUpdated report on the Oat Protein market ...
  8  https://bisouv.com/uncategorized/5101843/preim...  Ample Market Research’s new Market Intelligenc...,
  'e':                            

In [69]:
for count in countries:
    for cat in categories:
        master_dic[count][cat]['summary'] = np.nan
        master_dic[count][cat]['headline'] = np.nan
        for i in range(0,len(master_dic[count][cat]['text'])):
            try:
                if len(master_dic[count][cat]['text'][i]) > 300:
                    try:
                        aaa = summarizer(master_dic[count][cat]['text'][i][:2000], max_length=400, min_length=100, do_sample=False)
                        bbb = summarizer(aaa[0]['summary_text'], max_length=25, min_length=10, do_sample=False)
                        master_dic[count][cat]['summary'][i] = aaa[0]['summary_text']
                        master_dic[count][cat]['headline'][i] = bbb[0]['summary_text']
                    except:
                        print('*********** {0}****{1}'.format(count,cat))
            except: print('*****************************************************')
        master_dic[count][cat].dropna(inplace=True)
        master_dic[count][cat].reset_index(inplace=True,drop=True)

Your max_length is set to 400, but you input_length is only 172. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


*****************************************************


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if s

*****************************************************


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if s

In [70]:
master_dic

{'CA': {'b':                                                    0  ...                                           headline
  0  https://www.foxbusiness.com/business-leaders/e...  ...   Holmes faces multiple counts of mail fraud fo...
  1  https://ksusentinel.com/2021/05/23/home-elevat...  ...   Latest research on Global Home Elevator Marke...
  2  http://www.stocksequity.com/active-stocks/blus...  ...   Shares of Vaxart, Inc. (NASDAQ:VXRT) reached ...
  3  https://www.francetvinfo.fr/sciences/espace/es...  ...   Virgin Galactic is lancer un nouveau vol subo...
  4  https://www.baytoday.ca/local-news/need-beer-o...  ...   The Beer Store says 75 select locations acros...
  
  [5 rows x 4 columns],
  'e':                                                     0  ...                                           headline
  0   https://www.hollywoodreporter.com/news/music-n...  ...   The Weeknd named top male artist, top radio s...
  1   https://www.lapresse.ca/arts/television/2021-0...  ...   Radi

In [80]:
master_dic['CA']['b']

,0,text,summary,headline
0,https://www.foxbusiness.com/business-leaders/e...,Jurors will be able to hear limited evidence o...,Jurors will be able to hear limited evidence ...,Holmes faces multiple counts of mail fraud fo...
1,https://ksusentinel.com/2021/05/23/home-elevat...,Latest research on Global Home Elevator Market...,Latest research on Global Home Elevator Marke...,Latest research on Global Home Elevator Marke...
2,http://www.stocksequity.com/active-stocks/blus...,"Shares of Vaxart, Inc. (NASDAQ:VXRT) reached a...","Shares of Vaxart, Inc. (NASDAQ:VXRT) reached ...","Shares of Vaxart, Inc. (NASDAQ:VXRT) reached ..."
3,https://www.francetvinfo.fr/sciences/espace/es...,Une nouvelle étape est franchie vers le touris...,Virgin Galactic is lancer un nouveau vol subo...,Virgin Galactic is lancer un nouveau vol subo...
4,https://www.baytoday.ca/local-news/need-beer-o...,The Beer Store says 75 select locations across...,The Beer Store says 75 select locations acros...,The Beer Store says 75 select locations acros...


In [89]:
# !pip install kora -q
# !pip install bing-image-downloader
from kora.selenium import wd
from bs4 import BeautifulSoup
import time
import re
import requests
def image_download(query,path='/dataset/',max=5,show=True):
    try:
        wd.get("https://www.google.com/search?q={0}&rlz=1C1CHBF_enIN922IN922&sxsrf=ALeKk0378KGZuab4G3o516deoY5ZqW-xbg:1620111097016&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiGmLyTuK_wAhVuwzgGHfChD8kQ_AUoAXoECAIQAw&biw=1366&bih=635".format(query))
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        if max >= 75:
            time.sleep(5)
            wd.execute_script("window.scrollTo(15, document.body.scrollHeight);")
            time.sleep(5)
        page = wd.page_source
        soup = BeautifulSoup(page, 'html.parser')
        # soup.find_all(re.compile("(?:http\:|https\:)?\/\/.*\.(?:jpg)"))
        obj = re.findall('(?:http\:|https\:)?\/\/.*\.(?:jpg)',str(soup))
        print(len(obj))
        directory = os.getcwd()+'{0}{1}'.format(path,query)
        if not os.path.isdir(directory):
            os.mkdir(directory)
        for i in range(3,max+3):
            if show == True:
                print('downloading img no ',i-3)
            response = requests.get(obj[i])
            file = open("{0}/{1}{2}.jpg".format(directory,query,i), "wb")
            file.write(response.content)
            file.close()
        print('download {} images'.format(max))
    except OSError as error: 
        print(error)

In [95]:
try:
    os.mkdir('output')
except:
    print('already there')
fol = ['img','video','final_video','dataset']
for count in countries:
    for cat in categories:
        dfx = master_dic[count][cat]
        for y in range(0,len(fol)):
            a = fol[y]
            try:
                shutil.rmtree(a)
            except:
                print('error folder delete {}'.format(a))
            try:
                os.mkdir(a)
            except:
                print('error folder {}'.format(a))
        dfx = dfx.head(5)
        for i in range(0,len(dfx)):
            input_text = dfx.headline[i] + '!!!,' + dfx.summary[i]
            mels, alignment_history, audios = do_synthesis(input_text, tacotron2, melgan_stft, "TACOTRON", "MELGAN-STFT")
            scipy.io.wavfile.write('audios.mp3', rate=22050, data=audios)
            audio_clip = AudioFileClip('/content/audios.mp3')
            # time.sleep(5)
            try:
                image_download(dfx.headline[i],max=10)
            except:
                print('image download error ',dfx.headline[i])
            # downloader.download(dfx.headline[i], limit=10,  output_dir='dataset', adult_filter_off=True, force_replace=False, timeout=60)
            try:
                this_dir = os.listdir('/content/dataset/{}'.format(dfx.headline[i]))
                filepaths = [os.path.join('/content/dataset/{}'.format(dfx.headline[i]), fname) for fname in this_dir if fname.endswith("jpg")]
            except:
                print('image folder error')
            dir = '/content/img'
            for f in os.listdir(dir):
                os.remove(os.path.join(dir, f))
            for j in filepaths:
                try:
                    image = cv2.imread(j, 1)
                    stretch_near = cv2.resize(image, (1280, 720),interpolation = cv2.INTER_NEAREST)
                    cv2.imwrite('/content/img/{}'.format(j[-11:]), stretch_near)
                except:
                    print('resize error')
            if len(glob.glob('/content/img/*.jpg')) > 0:
                this_dir = os.listdir('/content/img')
                filepaths = [os.path.join('/content/img', fname) for fname in this_dir if fname.endswith("jpg")][:5]
                output_video = os.path.join('/content/video', 'video{}.mp4'.format(i))
                # clip = ImageSequenceClip(filepaths, fps=0.2)
                clip = ImageSequenceClip(filepaths, fps=0.25)
                clip.write_videofile(output_video, codec='libx264', audio_codec="aac")

                final_video_path = os.path.join('/content/final_video', 'final-video{}.mp4'.format(i))
                video_clip = VideoFileClip(output_video)

                final_clip = video_clip.set_audio(audio_clip)
                final_clip.write_videofile(final_video_path, codec='libx264', audio_codec="aac")

                tclip = VideoFileClip(final_video_path)

                out = ''
                wrapper = textwrap.TextWrapper(width=18)
                word_list = wrapper.wrap(text=dfx.headline[i])
                for element in word_list:
                    out = out + element + '\n'

                # Generate a text clip
                txt_clip = TextClip(out, fontsize = 65, color = 'black', font="Amiri-Bold")
                    
                # setting position of text in the center and duration will be 10 seconds
                txt_clip = txt_clip.set_pos('center',relative=True).set_duration(5)
                    
                # Overlay the text clip on the first video clip
                video = CompositeVideoClip([tclip, txt_clip])
                video.write_videofile(final_video_path, codec='libx264', audio_codec="aac")
            else:
                print('No Images Found '*5)

        clipss = {}
        for z in range(0,len(glob.glob('/content/final_video/*.mp4'))):
            clipss['c{}'.format(z)] = VideoFileClip(glob.glob('/content/final_video/*.mp4')[z])
        clips = []
        for state, capital in clipss.items():
            clips.append(capital)
        final = concatenate_videoclips(clips)
        final_video_path = os.path.join('/content/output/', '{}final-video.mp4'.format(link[k]))
        final.write_videofile(final_video_path, codec='libx264', audio_codec="aac")

already there
35
downloading img no  0
downloading img no  1
downloading img no  2
downloading img no  3
downloading img no  4
downloading img no  5
downloading img no  6
downloading img no  7
downloading img no  8
downloading img no  9
download 10 images
[MoviePy] >>>> Building video /content/video/video0.mp4
[MoviePy] Writing video /content/video/video0.mp4


 83%|████████▎ | 5/6 [00:00<00:00, 50.97it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/video/video0.mp4 

[MoviePy] >>>> Building video /content/final_video/final-video0.mp4
[MoviePy] Writing audio in final-video0TEMP_MPY_wvf_snd.mp4


100%|██████████| 1025/1025 [00:02<00:00, 464.65it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/final_video/final-video0.mp4



 83%|████████▎ | 5/6 [00:00<00:00, 124.13it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/final_video/final-video0.mp4 



OSError: ignored

In [96]:
FFMPEG_BINARY = os.getenv('FFMPEG_BINARY', 'ffmpeg-imageio')
# IMAGEMAGICK_BINARY = os.getenv('IMAGEMAGICK_BINARY', 'auto-detect')
# Modify to the installation path of ImageMagic just now
IMAGEMAGICK_BINARY = r"D:\python_study_tools\ImageMagick-7.0.9-Q16\magick.exe"  
